In [17]:
import pandas as pd
import numpy as np


In [18]:
df = pd.read_csv(r'C:\Users\pablo\Desktop\proyecto_scoring_gmsc\give me some credit\data\raw\cs-training.csv')

In [19]:
df = df.iloc[:,1:]

In [20]:
# Winsorización de RUUL: valores extremos (hasta 50k) eran errores de carga. Se limita al p99 (~1.09)
p99_ruul = df['RevolvingUtilizationOfUnsecuredLines'].quantile(0.99)
df['RevolvingUtilizationOfUnsecuredLines'] = df['RevolvingUtilizationOfUnsecuredLines'].clip(0, p99_ruul)

In [21]:
# Winsorización de 'age': valores imposibles (0 años) y máximos irreales (109). Se limita entre 18 y p99 (~87).
p99_age = df['age'].quantile(0.99)
df['age'] = df['age'].clip(lower=18, upper=p99_age)

In [22]:
# Winsorización de moras 30–59 días: valores extremos (hasta 98) eran inconsistentes con la edad. Se limita al p99 (~4).
p99_3059 = df['NumberOfTime30-59DaysPastDueNotWorse'].quantile(0.99)
df['NumberOfTime30-59DaysPastDueNotWorse'] = df['NumberOfTime30-59DaysPastDueNotWorse'].clip(upper=p99_3059)

In [23]:
# Limpieza de MonthlyIncome: los valores 0 eran inválidos, se reemplazan por NaN y se imputan con la mediana (~5400).
df['MonthlyIncome'] = df['MonthlyIncome'].replace(0, np.nan)
df['MonthlyIncome'] = df['MonthlyIncome'].fillna(df['MonthlyIncome'].median())

In [24]:
# Winsorización de MonthlyIncome: valores extremos (hasta 3M) eran errores de carga. Se limita al p99 (~23k).
p99_inc = df['MonthlyIncome'].quantile(0.99)
df['MonthlyIncome'] = df['MonthlyIncome'].clip(0, p99_inc)

In [25]:
# valores muy bajos (<1000) son ingresos imposibles (errores/missings). Se reemplazan por NaN.
df['MonthlyIncome'] = df['MonthlyIncome'].mask(df['MonthlyIncome'] < 1000, np.nan)

In [26]:
# Imputación con la mediana (~5400)
df['MonthlyIncome'] = df['MonthlyIncome'].fillna(df['MonthlyIncome'].median())

In [27]:
# Winsorización de DebtRatio: outliers absurdos (>300k). Se limita a 0–2 (rango realista).
df['DebtRatio'] = df['DebtRatio'].clip(0, 2)

In [28]:
# Winsorización de líneas de crédito: valores máximos (58) son atípicos.
# Se limita al p99 (~24), rango razonable para historial crediticio.
p99_open = df['NumberOfOpenCreditLinesAndLoans'].quantile(0.99)
df['NumberOfOpenCreditLinesAndLoans'] = df['NumberOfOpenCreditLinesAndLoans'].clip(0, p99_open)

In [ ]:
# Corrección de NumberOfTimes90DaysLate: valores 96–98 son errores de carga (placeholders de missing).
# Se reemplazan por 0 y se limita a un rango razonable (0–5).
df['NumberOfTimes90DaysLate'] = df['NumberOfTimes90DaysLate'].mask(df['NumberOfTimes90DaysLate'] >= 90, np.nan)
df['NumberOfTimes90DaysLate'] = df['NumberOfTimes90DaysLate'].fillna(0)
df['NumberOfTimes90DaysLate'] = df['NumberOfTimes90DaysLate'].clip(0, 5)



In [39]:
# Winsorización de hipotecas: valores >4 son errores de carga.
# Se limita al p99 (~4), preservando la señal sin distorsionar la distribución.
p99 = df['NumberRealEstateLoansOrLines'].quantile(0.99)
df['NumberRealEstateLoansOrLines'] = df['NumberRealEstateLoansOrLines'].clip(0, p99)

In [ ]:
# Corrección de atrasos 60-89 días: valores >=90 (96-98) son errores de carga.
# Se reemplazan por NaN y se imputan con 0, manteniendo la distribución real.
df['NumberOfTime60-89DaysPastDueNotWorse'] = df['NumberOfTime60-89DaysPastDueNotWorse'].mask(
    df['NumberOfTime60-89DaysPastDueNotWorse'] >= 90, np.nan)
df['NumberOfTime60-89DaysPastDueNotWorse'] = df['NumberOfTime60-89DaysPastDueNotWorse'].fillna(0)

In [48]:
# Outliers absurdos (hasta 20). Se limita a un máximo razonable (5)
df['NumberOfDependents'] = df['NumberOfDependents'].clip(0, 5)

# Falta de datos: NaN se imputan como 0 (cliente sin dependientes)
df['NumberOfDependents'] = df['NumberOfDependents'].fillna(0)


In [52]:
df.to_csv(r'C:\Users\pablo\Desktop\proyecto_scoring_gmsc\give me some credit\data\clean\gmsc_clean.csv', index=False)
